# Check MinIO

In [2]:
!pip install minio delta-spark==2.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 1.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 3.4 MB/s eta 0:00:00a 0:00:01


In [3]:
from minio import Minio

In [4]:
client = Minio(
    "minio:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False
)

bucket = "warehouse"
if client.bucket_exists(bucket):
    print(f"{bucket} exists")


warehouse exists


# Init SparkContext

In [1]:
from datetime import datetime
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext

In [2]:
spark = (SparkSession.builder.appName("pyspark-rdd-demo-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())

sqlContext = SQLContext(spark)
spark.sparkContext.getConf().getAll()


/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[('spark.repl.local.jars',
  'file:///usr/local/spark-3.3.1-bin-hadoop3/jars/delta-core_2.12-2.2.0.jar,file:///usr/local/spark-3.3.1-bin-hadoop3/jars/hadoop-aws-3.3.2.jar,file:///usr/local/spark-3.3.1-bin-hadoop3/jars/delta-storage-2.2.0.jar,file:///usr/local/spark-3.3.1-bin-hadoop3/jars/aws-java-sdk-1.12.367.jar,file:///usr/local/spark-3.3.1-bin-hadoop3/jars/s3-2.18.41.jar,file:///usr/local/spark-3.3.1-bin-hadoop3/jars/aws-java-sdk-bundle-1.11.1026.jar'),
 ('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false'),
 ('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.conc

In [3]:
sc = spark.sparkContext
sc

<SparkContext master=spark://spark-master:7077 appName=pyspark-rdd-demo-2023-01-25 08:52:41.683891>

In [4]:
# test Delta Lake
spark.range(500).write.format("delta").save("s3a://warehouse/deltafile", mode="overwrite")

# Create RDDs

## By loading dataset

In [5]:
fdd = sc.textFile("s3a://warehouse/testfile.txt")
fdd

s3a://warehouse/testfile.txt MapPartitionsRDD[25] at textFile at NativeMethodAccessorImpl.java:0

In [6]:
fdd.collect()

['',
 'What is Lorem Ipsum?',
 '',
 "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.",
 'Why do we use it?',
 '',
 "It is a long established fact that a reader will be distracted by the readable content of a page when looking at its layout. The point of using Lorem Ipsum is that it has a more-or-less normal distribution of letters, as opposed to using 'Content here, content here', making it look like readable English. Many desktop publishing packages and web page edito

## By using parallelize

# RDD operations

# Caching RDDs